**This notebook analyzes actions taken by players given specific states. (Last updated: 30 May 2023)**

In [ ]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import numpy as np
import json

Our first step is to extract state-action pairs for each player in each round. This also introduces a way to clean the raw dataset (i.e. contains metadata/irrelavant fields) for any future analysis tasks.

In [ ]:
# Reads in step_data.csv, which contains all timesteps of all users
# Cleans the raw data, preserving only non-empty rows/responses
step_data = pd.read_csv("step_data.csv")
step_data = step_data[(~step_data['round_0_states'].isna()) & (~step_data['round_1_states'].isna()) & (~step_data['round_2_states'].isna()) & (~step_data['round_3_states'].isna()) & (~step_data['round_4_states'].isna()) & (~step_data['round_5_states'].isna()) & (~step_data['round_6_states'].isna())]
step_data

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,round_6_actions,round_6_states,round_6_score,round_6_reward,round_6_bonus,round_6_elapsed_time,MID,treatment,tipcondition,intervention
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,round_6_actions,round_6_states,round_6_score,round_6_reward,round_6_bonus,round_6_elapsed_time,MID,treatment,tipcondition,intervention
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""round_6_actions""}","{""ImportId"":""round_6_states""}","{""ImportId"":""round_6_score""}","{""ImportId"":""round_6_reward""}","{""ImportId"":""round_6_bonus""}","{""ImportId"":""round_6_elapsed_time""}","{""ImportId"":""MID""}","{""ImportId"":""treatment""}","{""ImportId"":""tipcondition""}","{""ImportId"":""intervention""}"
28,2022-09-11 23:15:43,2022-09-12 00:04:19,IP Address,108.74.160.181,100,2915,True,2022-09-12 00:04:19,R_2ypgoh2f2tpygT7,NaN,...,"[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...",NaN,4,0,44,NaN,nochef,t1,paysocial
29,2022-09-12 11:40:19,2022-09-12 11:46:55,IP Address,108.74.160.181,100,395,True,2022-09-12 11:46:56,R_0rpSmV5kSUphP8Z,NaN,...,"[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...",NaN,4,0,48,NaN,nochef,t1,social
35,2022-09-12 15:37:51,2022-09-12 15:58:05,IP Address,85.9.204.121,100,1214,True,2022-09-12 15:58:06,R_1N9BntMC65Uo6Jd,NaN,...,"[[{""order_index"":1,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...",NaN,4,0.2,39,NaN,nochef,t1,pay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,2022-09-13 20:30:01,2022-09-13 20:54:07,IP Address,208.195.175.135,100,1445,True,2022-09-13 20:54:07,R_3L0eFe04liYrJ6e,NaN,...,"[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...",NaN,4,0.2,39,NaN,nochef,t1,curriculum
195,2022-09-13 21:15:50,2022-09-13 22:00:07,IP Address,208.195.160.130,100,2657,True,2022-09-13 22:00:08,R_1OTgTd7lB0VIpck,NaN,...,"[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...",NaN,4,0.2,38,NaN,nochef,t1,control
196,2022-09-13 21:51:55,2022-09-13 22:07:52,IP Address,208.200.1.199,100,956,True,2022-09-13 22:07:52,R_2TyFINmYq604WBc,NaN,...,"[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...",NaN,4,0,44,NaN,nochef,t1,curriculum
197,2022-09-13 21:40:50,2022-09-13 22:16:18,IP Address,107.180.131.175,100,2128,True,2022-09-13 22:16:20,R_2ygtRSgq9C70cKc,NaN,...,"[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...",NaN,4,0.6,37,NaN,nochef,t1,paysocial


In [ ]:
# Extract state-action pairs in each round {0, 1, ..., 6}
state_action = step_data[['ResponseId', 'round_0_actions', 'round_0_states', 'round_1_actions', 'round_1_states',
                          'round_2_actions', 'round_2_states', 'round_3_actions', 'round_3_states',
                          'round_4_actions', 'round_4_states', 'round_5_actions', 'round_5_states',
                          'round_6_actions', 'round_6_states']]
state_action = state_action.iloc[2:].reset_index().drop('index', axis=1)
state_action.head()

,ResponseId,round_0_actions,round_0_states,round_1_actions,round_1_states,round_2_actions,round_2_states,round_3_actions,round_3_states,round_4_actions,round_4_states,round_5_actions,round_5_states,round_6_actions,round_6_states
0,R_2ypgoh2f2tpygT7,"[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":99,""max_ticks"":10,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":40,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":40,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord..."
1,R_0rpSmV5kSUphP8Z,"[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":99,""max_ticks"":10,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":40,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":40,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord..."
2,R_1N9BntMC65Uo6Jd,"[[{""order_index"":0,""task"":2,""worker_index"":0},...","[{""params"":{""config_id"":99,""max_ticks"":10,""ord...","[[{""order_index"":2,""task"":0,""worker_index"":1},...","[{""params"":{""config_id"":40,""max_ticks"":50,""ord...","[[{""order_index"":2,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":40,""max_ticks"":50,""ord...","[[{""order_index"":2,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":2,""task"":0,""worker_index"":1}]...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":1,""task"":0,""worker_index"":1}]...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":1,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord..."
3,R_tEAxrJOsPTNSNz3,"[[{""order_index"":2,""task"":0,""worker_index"":2},...","[{""params"":{""config_id"":99,""max_ticks"":10,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":40,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":40,""max_ticks"":50,""ord...","[[{""order_index"":1,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":1,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":2,""task"":0,""worker_index"":1},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord...","[[{""order_index"":1,""task"":0,""worker_index"":1},...","[{""params"":{""config_id"":42,""max_ticks"":50,""ord..."
4,R_3HH7y0bGp3SJMO7,"[[{""order_index"":0,""task"":2,""worker_index"":0},...","[{""params"":{""config_id"":99,""max_ticks"":10,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":0},...","[{""params"":{""config_id"":40,""max_ticks"":50,""ord...","[[{""order_index"":0,""task"":0,""worker_index"":1},...","[{""params"":{""config_id"":40,""max_ticks"":50,""ord...","[[{""order_index"":2,""task"":0,""worker_in

In [ ]:
column_names = [
    'round_0_actions', 'round_0_states', 
    'round_1_actions', 'round_1_states',
    'round_2_actions', 'round_2_states', 
    'round_3_actions', 'round_3_states',
    'round_4_actions', 'round_4_states', 
    'round_5_actions', 'round_5_states',
    'round_6_actions', 'round_6_states'
]

for column in column_names:
  state_action[column] = state_action[column].apply(json.loads)

In [ ]:
table_list = []

for i in range(7):
  round_x_actions = state_action[f'round_{i}_actions'].explode()
  round_x_states = state_action[f'round_{i}_states'].explode()
  temp = pd.DataFrame({'actions': round_x_actions, 'states': round_x_states}).reset_index(drop=True)
  temp['actions'] = temp['actions'].apply(lambda x: x[0] if len(x) > 0 else None)
  temp = temp.dropna().reset_index(drop=True)
  temp['order_index'] = temp['actions'].apply(lambda x: x['order_index'])
  temp['task'] = temp['actions'].apply(lambda x: x['task'])
  temp['worker_index'] = temp['actions'].apply(lambda x: x['worker_index'])
  temp['orders & workers'] = temp['states'].apply(lambda x: (x['params']['orders'], x['params']['workers']))
  temp = temp.drop(columns=['states', 'actions'])
  table_list.append(temp)

In [ ]:
result = pd.concat(table_list)
result

,order_index,task,worker_index,orders & workers
0,0,0,0,"([taco, taco, taco, taco], [chef, chef, chef])"
1,0,1,0,"([taco, taco, taco, taco], [chef, chef, chef])"
2,0,2,0,"([taco, taco, taco, taco], [chef, chef, chef])"
3,0,3,0,"([taco, taco, taco, taco], [chef, chef, chef])"
4,0,0,0,"([taco, taco, taco, taco], [chef, chef, chef])"
...,...,...,...,...
1041,2,1,0,"([burger, burger, burger, burger], [sous chef,..."
1042,2,2,0,"([burger, burger, burger, burger], [sous chef,..."
1043,3,2,1,"([burger, burger, burger, burger], [sous chef,..."
1044,1,0,0,"([burger, burger, burger, burger], [sous chef,..."


In [ ]:
state_action.to_csv('state_action.csv')

Another approach I tried earlier was to use the data generator: 

In [ ]:
action_summary = pd.read_csv('action_summary.csv')
action_summary.head()

,Unnamed: 0,ResponseId,intervention,round,tick,worker,order,task,chef_round_chopping,souschef_round_chopping,server_round_chopping,chef_round_cooking,souschef_round_cooking,server_round_cooking,chef_round_plating,souschef_round_plating,server_round_plating
0,0,R_2ypgoh2f2tpygT7,"[[{""order_index"":0,""task"":0,""worker_index"":0},...",1,1,0,0,0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0
1,1,R_2ypgoh2f2tpygT7,"[[{""order_index"":0,""task"":0,""worker_index"":0},...",1,1,1,1,0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0
2,2,R_2ypgoh2f2tpygT7,"[[{""order_index"":0,""task"":0,""worker_index"":0},...",1,1,2,2,0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0
3,6,R_2ypgoh2f2tpygT7,"[[{""order_index"":0,""task"":0,""worker_index"":0},...",1,3,0,0,1,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0
4,10,R_2ypgoh2f2tpygT7,"[[{""order_index"":0,""task"":0,""worker_index"":0},...",1,4,1,1,1,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0


In [ ]:
action_brief = action_summary[['ResponseId', 'round', 'tick', 'worker', 'order']]
action_brief.head(30)

,ResponseId,round,tick,worker,order
0,R_2ypgoh2f2tpygT7,1,1,0,0
1,R_2ypgoh2f2tpygT7,1,1,1,1
2,R_2ypgoh2f2tpygT7,1,1,2,2
3,R_2ypgoh2f2tpygT7,1,3,0,0
4,R_2ypgoh2f2tpygT7,1,4,1,1
5,R_2ypgoh2f2tpygT7,1,5,2,2
6,R_2ypgoh2f2tpygT7,1,8,0,0
7,R_2ypgoh2f2tpygT7,1,14,1,1
8,R_2ypgoh2f2tpygT7,1,16,0,3
9,R_2ypgoh2f2tpygT7,1,18,0,3


In [ ]:
state = pd.read_csv('state.csv')
state.head(30)

,Unnamed: 0,ID,round,chef_order,chef_task,chef_left_ticks,sou_chef_order,sou_chef_task,sou_chef_left_ticks,server_order,...,order0_task2,order1_task0,order1_task1,order1_task2,order2_task0,order2_task1,order2_task2,order3_task0,order3_task1,order3_task2
0,0,R_2ypgoh2f2tpygT7,round_1_states,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,unavailable,available,unavailable,unavailable,available,unavailable,unavailable,available,unavailable,unavailable
1,1,R_2ypgoh2f2tpygT7,round_1_states,0.0,0.0,1.0,1.0,0.0,2.0,2.0,...,unavailable,sou_chef,unavailable,unavailable,server,unavailable,unavailable,available,unavailable,unavailable
2,2,R_2ypgoh2f2tpygT7,round_1_states,NaN,NaN,NaN,1.0,0.0,1.0,2.0,...,unavailable,sou_chef,unavailable,unavailable,server,unavailable,unavailable,available,unavailable,unavailable
3,3,R_2ypgoh2f2tpygT7,round_1_states,0.0,1.0,4.0,NaN,NaN,NaN,2.0,...,unavailable,finished,available,unavailable,server,unavailable,unavailable,available,unavailable,unavailable
4,4,R_2ypgoh2f2tpygT7,round_1_states,0.0,1.0,3.0,1.0,1.0,8.0,NaN,...,unavailable,finished,sou_chef,unavailable,finished,available,unavailable,available,unavailable,unavailable
5,5,R_2ypgoh2f2tpygT7,round_1_states,0.0,1.0,2.0,1.0,1.0,7.0,2.0,...,unavailable,finished,sou_chef,unavailable,finished,server,unavailable,available,unavailable,unavailable
6,6,R_2ypgoh2f2tpygT7,round_1_states,0.0,1.0,1.0,1.0,1.0,6.0,2.0,...,unavailable,finished,sou_chef,unavailable,finished,server,unavailable,available,unavailable,unavailable
7,7,R_2ypgoh2f2tpygT7,round_1_states,NaN,NaN,NaN,1.0,1.0,5.0,2.0,...,available,finished,sou_chef,unavailable,finished,server,unavailable,available,unavailable,unavailable
8,8,R_2ypgoh2f2tpygT7,round_1_states,0.0,2.0,6.0,1.0,1.0,4.0,2.0,...,chef,finished,sou_chef,unavailable,finished,server,unavailable,available,unavailable,unavailable
9,9,R_2ypgoh2f2tpygT7,round_1_states,0.0,2.0,5.0,1.0,1.0,3.0,2.0,...,chef,finished,sou_chef,unavailable,finished,server,unavailable,available,unavailable,unavailable


In [ ]:
# ResponseId = 'R_2ypgoh2f2tpygT7'
# round = 1
# tick = 1

results = []
# For every responseId, look up the state at tick t from state_actions table
for ResponseId in state_action['ResponseId']: # all valid ResponseIDs
  for round in np.arange(0, 7): # all valid rounds
    # Retrieve state information for (ResponseId, round) pair
    specific_states = list(state_action[state_action['ResponseId'] == ResponseId][f'round_{round}_states'])[0]
    specific_states = json.loads(specific_states)
    for one_state in specific_states:
      workers = one_state['params']['workers']
      orders = one_state['params']['orders']
      tick = one_state['ticks']
      res = (ResponseId, round, tick, workers, orders)
      if res not in results:
        results.append(res)


In [ ]:
state_info = pd.DataFrame(results, columns=['ResponseId', 'round', 'tick', 'workers', 'orders'])
state_info.head(10)

,ResponseId,round,tick,workers,orders
0,R_2ypgoh2f2tpygT7,0,0,"[chef, chef, chef]","[taco, taco, taco, taco]"
1,R_2ypgoh2f2tpygT7,0,1,"[chef, chef, chef]","[taco, taco, taco, taco]"
2,R_2ypgoh2f2tpygT7,0,2,"[chef, chef, chef]","[taco, taco, taco, taco]"
3,R_2ypgoh2f2tpygT7,0,3,"[chef, chef, chef]","[taco, taco, taco, taco]"
4,R_2ypgoh2f2tpygT7,0,4,"[chef, chef, chef]","[taco, taco, taco, taco]"
5,R_2ypgoh2f2tpygT7,0,5,"[chef, chef, chef]","[taco, taco, taco, taco]"
6,R_2ypgoh2f2tpygT7,0,6,"[chef, chef, chef]","[taco, taco, taco, taco]"
7,R_2ypgoh2f2tpygT7,0,7,"[chef, chef, chef]","[taco, taco, taco, taco]"
8,R_2ypgoh2f2tpygT7,0,8,"[chef, chef, chef]","[taco, taco, taco, taco]"
9,R_2ypgoh2f2tpygT7,1,0,"[chef, sous chef, server]","[burger, burger, burger, burger]"


In [ ]:
action_brief.merge(state_info, on=['ResponseId', 'round', 'tick'])

,ResponseId,round,tick,worker,order,workers,orders
0,R_2ypgoh2f2tpygT7,1,1,0,0,"[chef, sous chef, server]","[burger, burger, burger, burger]"
1,R_2ypgoh2f2tpygT7,1,1,1,1,"[chef, sous chef, server]","[burger, burger, burger, burger]"
2,R_2ypgoh2f2tpygT7,1,1,2,2,"[chef, sous chef, server]","[burger, burger, burger, burger]"
3,R_2ypgoh2f2tpygT7,1,3,0,0,"[chef, sous chef, server]","[burger, burger, burger, burger]"
4,R_2ypgoh2f2tpygT7,1,4,1,1,"[chef, sous chef, server]","[burger, burger, burger, burger]"
...,...,...,...,...,...,...,...
5393,R_3m8YUt51nvAaB4p,6,21,2,3,"[sous chef, server]","[burger, burger, burger, burger]"
5394,R_3m8YUt51nvAaB4p,6,33,1,2,"[sous chef, server]","[burger, burger, burger, burger]"
5395,R_3m8YUt51nvAaB4p,6,37,2,3,"[sous chef, server]","[burger, burger, burger, burger]"
5396,R_3m8YUt51nvAaB4p,6,42,1,1,"[sous chef, server]","[burger, burger, burger, burger]"


# Previous frequency count: naive version


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Thunderbeee/hai-cooking/main/data/Archive/combined_phase2.csv")
df.head()

,ResponseId,round,tick,orderId,taskId,taskTicks,workerId,workerTicks,best_tip
0,R_0DNcJqALC3UG9Db,1,1,1,1,2,1,0,2
1,R_0DNcJqALC3UG9Db,1,1,2,1,2,2,0,2
2,R_0DNcJqALC3UG9Db,1,1,3,1,2,3,0,2
3,R_0DNcJqALC3UG9Db,1,2,0,0,0,0,0,2
4,R_0DNcJqALC3UG9Db,1,2,0,0,0,0,0,2


In [ ]:
temp = df[(df['ResponseId'] == "R_0DNcJqALC3UG9Db") & (df['round'] == 3)]
sum((temp['workerId'] == 1) & (temp['taskId'] == 2))

3

In [ ]:
# TODO: to customize this func based on tips, need a mapping of tips and 
# TODO: figure out which group each player belongs to?
# TODO: workerid <-> worker role

# state is what tasks are available and what workers are available

def compute_frequency(df, tip = -1):
  """
  Given a specific tip, returns the frequency count of players who follow the tip.
  df: A pandas DataFrame containing 
  tip: The tip for which we want to compute the freq count. Default is -1 (servers cook twice?)

  Referenced Jeremy's compliance analysis: https://github.com/Thunderbeee/hai-cooking/blob/main/step%20model:compliance/analysis/tip_dim1_analysis.ipynb
  """
  player_ids = []
  rounds = []
  strats = []

  for player in set(df['ResponseId']):
    comply = False
    for round in range(1, 7):
      temp = df[(df['ResponseId'] == player) & (df['round'] == round)]
      # match tip:
      #   case -1:
      player_ids.append(player)
      rounds.append(round)
      if round <= 2:
        numServerCooks = sum((temp['workerId'] == 2) & (temp['taskId'] == 2))
      else:
        numServerCooks = sum((temp['workerId'] == 1) & (temp['taskId'] == 2))
      
      if numServerCooks == 2:
        comply = True
      else:
        comply = False
    strats.extend([comply] * 6) # it only matters whether the player followed the provided tip throughout the game

  res = {
      "ResponseId": player_ids,
      "round": rounds,
      "compliance": strats
  }
  
  res = pd.DataFrame(res)
  # print(res)
  avg = res.groupby("ResponseId").agg(np.average).reset_index().loc[:, ["ResponseId", "compliance"]]
  return avg
      

In [ ]:
compute_frequency(df)

             ResponseId  round  compliance
0     R_3sc5QXqKfbXcthq      1       False
1     R_3sc5QXqKfbXcthq      2       False
2     R_3sc5QXqKfbXcthq      3       False
3     R_3sc5QXqKfbXcthq      4       False
4     R_3sc5QXqKfbXcthq      5       False
...                 ...    ...         ...
1441  R_3EF2jj6VqfT4YVK      2       False
1442  R_3EF2jj6VqfT4YVK      3       False
1443  R_3EF2jj6VqfT4YVK      4       False
1444  R_3EF2jj6VqfT4YVK      5       False
1445  R_3EF2jj6VqfT4YVK      6       False

[1446 rows x 3 columns]


,ResponseId,compliance
0,R_0DNcJqALC3UG9Db,0.0
1,R_0HbfbDtnkUIRbvH,1.0
2,R_0cVkkjfJMy8V6P7,0.0
3,R_10OZ3Fgz4TJo8Vx,0.0
4,R_11WSDnMOeKXakQH,0.0
...,...,...
236,R_xlnJEPOkxxHspJT,0.0
237,R_yNJaIImQ7EY3J5v,0.0
238,R_yqY6xKo4dUHco6d,0.0
239,R_yrKr3inrXDL5Lvb,0.0
